# Library

In [3]:
pip install selectivesearch

  Preparing metadata (setup.py) ... done
  Created wheel for selectivesearch: filename=selectivesearch-0.4-py3-none-any.whl size=4335 sha256=4294cbd37e8f8425cf832d1f6e22c4900f4d836f63a1b3daf6e78e68b118a435
  Stored in directory: /root/.cache/pip/wheels/0e/49/95/01447a4e0f48a135ac91fbdb1dd2a1c0523e40e29957b383a3
Successfully built selectivesearch
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imageio.v3 as imageio
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import pandas as pd
import cv2
import shutil
import json
import yaml
import random
import time
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import concurrent.futures
import multiprocessing as mp
from PIL import Image, ImageOps
from collections import defaultdict, Counter
from torchvision import transforms
from torchvision.transforms import functional as TF
import torch.optim as optim
import re
import selectivesearch
import torch.optim as optim
from torchvision import models
from torchvision.models import AlexNet_Weights
import matplotlib.patches as mpatches
import torch.nn.functional as F
from sklearn.svm import SVC

# Path

In [9]:
#Data sources
IN_DATASET_NM = 'our-xview-dataset'

#Output folders and file names
OUT_COCO_JSON_NM = 'COCO_annotations_new.json'
OUT_IMAGE_FLDR_NM = 'images'
OUT_CFG_FLDR_NM = 'YOLO_cfg'
OUT_DATAFRAME_NM = 'xview_labels.parquet'
YAML_NM = 'xview_yolo.yaml'
RANDOM_SEED = 2023

in_dataset_pth = Path('/kaggle/our-input/xview-dataset')
out_dataset_pth = Path('/kaggle/working/')
future_ds_img_fldr = Path(f'/kaggle/working/{OUT_IMAGE_FLDR_NM}')
future_ds_cfg_fldr = Path(f'/kaggle/working/{OUT_CFG_FLDR_NM}')

out_data_parquet_pth = out_dataset_pth / OUT_DATAFRAME_NM
out_data_parquet_pth = out_dataset_pth / OUT_DATAFRAME_NM
coco_json_pth = out_dataset_pth / OUT_COCO_JSON_NM
yolo_yaml_pth = cfg_fldr_pth / YAML_NM
train_txt_pth = cfg_fldr_pth / 'train.txt'
val_txt_pth = cfg_fldr_pth / 'val.txt'
test_txt_pth = cfg_fldr_pth / 'test.txt'


def make_empty_dir(directory):
    if directory.is_dir():
        shutil.rmtree(directory)
    os.makedirs(directory)

make_empty_dir(cfg_fldr_pth)

random.seed(RANDOM_SEED)

# DataLoader

## Region Proposals Generation

In [ ]:
# le immagini devono essere aperte con Image.open
image = Image.open(img_path).convert('RGB') #apro l'immagine e ne ricavo le dimensioni
original_width, original_height = image.size

dir_name = #directory in cui salvare le region_proposal

def generate_dataset_proposals(txt_file, dir_name):
  # prendo i path delle immagini e li memorizzo in una lista
   with open(txt_file, 'r') as f:
            image_paths = [line.strip() for line in f.readlines()]

  img_dir = img_dir
  os.makedirs(img_dir, exist_ok=True)

  for index in range(len(image_paths)):
    img_name = os.path.basename(image_paths[index]) #prendo il path dell'immagine da self.image_paths in base all'indice fornito
    img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', '')) #ricavo l'id dell'immagine -> non l'ho già fatto nell'init?

    dir_image = os.path.join(dir_name, img_id) #nome della directory che conterrà le region proposals relative all'immagine

    if img_id not in self.image_info: #vedo dal dizionario self.image_info se l'immagine è contenuta nel file COCO
              raise ValueError(f"Immagine {img_name} non trovata nel file COCO")

    img_path = os.path.join(self.img_dir, img_name) #prendo il path completo dell'immagine unendo il path della cartella con il nome.jpg dell'immagine
    if not os.path.exists(img_path): #se il path non esiste allora lo segnalo
              raise ValueError(f"Immagine non trovata nel percorso: {img_path}")

    image = Image.open(img_path).convert('RGB') #apro l'immagine e ne ricavo le dimensioni
    original_width, original_height = image.size

     # GESTIONE DELLE REGION PROPOSALS
     proposals_tensor = generate_region_proposals(image) # image = immagine aperta in formato RGB con la libreria PIL
     #    produce una lista di proposals nel formato (x_min, y_min, x_max, y_max)

     processed_proposals = process_proposals(image_tensor, proposals_tensor) # image_tensor = image dopo la data agumentation
     #    produce le immagine = region proposals relative all'immagine di input

     # salvo le region proposals come immagini in una cartella relativa all'immagine di input
     # - salvataggio in dir_image
     os.makedirs(dir_image, exist_ok=True)

    # Iterare sulle region proposals e salvarle come immagini
    for i, proposal_tensor in enumerate(processed_proposals):
        # Convertire il tensore in immagine PIL (assumendo valori nel range [0, 1])
        proposal_image = Image.fromarray((proposal_tensor.numpy() * 255).astype('uint8'))

        # Generare un nome file unico
        proposal_filename = os.path.join(dir_image, f'proposal_{i:04d}.jpg')

        # Salvare l'immagine
        proposal_image.save(proposal_filename)

        print(f"Salvata proposal {i+1}/{len(processed_proposals)}: {proposal_filename}")

        # Aggiungi il path relativo alla lista
        relative_path = os.path.relpath(proposal_filename, dir_name)  # Path relativo rispetto a dir_name
        all_proposal_paths.append(relative_path)

    # Scrittura di tutti i path relativi in un unico file .txt
    with open(output_txt, 'w') as txt_file:
        for path in all_proposal_paths:
            txt_file.write(f"{path}\n")

    print(f"Creato file TXT con i path relativi di tutte le region proposals: {output_txt}")

def generate_region_proposals(image): #funzione per la generazione delle region proposals per singola immagine
        img_np = np.array(image) #trasformo l'immagine in un array numpy

        if len(img_np.shape) == 3 and img_np.shape[0] == 3: #porto l'immagine nel formato corretto
            img_np = np.transpose(img_np, (1, 2, 0))  # Da [C, H, W] a [H, W, C]

        _, regions = selectivesearch.selective_search(img_np, scale=500, sigma=0.9, min_size=10) #richiamo la funzione di selective search
        #scale: granularità della ricerca (più alto, meno dettagliato) ; sigma: Standard deviation per il filtro gaussiano usato per la segmentazione ;
        #min_size: Dimensione minima di un segmento nell'algoritmo
        #regions: lista di regioni candidate (proposals).
        # - regione = dizionario che contiene info. -> incluse le coordinate di un rettangolo delimitante (region['rect'])

        #CHECK SULLA PRODUZIONE DELLE REGION PROPOSALS
        if len(regions) == 0:
            print(f"Warning: Nessuna regione proposta generata per immagine con forma {img_np.shape}.")

        candidate_proposals = []
        for region in regions: #per ogni regione nella lista delle regioni candidate
            x, y, w, h = region['rect'] # prendo le coordinate del rettangolo delimitante
            if w > 0 and h > 0 and w >= 10 and h >= 10: # prendo solo le regioni con altezza e larghezza >= 10 per evitare che siano molto rumorose
                area = w * h
                x_max, y_max = min(x + w, img_np.shape[1]), min(y + h, img_np.shape[0]) # limito la regione alle dimensioni dell'immagine
                candidate_proposals.append([x, y, x_max, y_max, area]) #inserisco la nuova regione nella lista delle region proposals -> aggiungo un valore in più (area) per facilitare il filtraggio dopo

        unique_proposals = list(set(tuple(p) for p in candidate_proposals)) # converto le proposals in tuple in modo da eliminare i duplicati

        #in questo modo non viene preservata la corrispondenza tra region proposals e labels

        #FILTRO LE PROPOSALS PER PRENDERE SOLO QUELLE UTILI/NECESSARIE
        min_area = 10
        max_area_ratio = 0.8
        proposals = []

        for x_min, y_min, x_max, y_max, area in unique_proposals: #per ogni proposal
            if area >= min_area and area <= max_area_ratio * (img_width * img_height):
                proposals.append((x_min, y_min, x_max, y_max))

        return proposals # restituisce le region proposal valide

def process_proposals(image_tensor, proposals, output_size=(227, 227)): # la funzione trasforma le proposals trovate in immagini ottenute ritagliando l'imagine originale
        processed_proposals = []
        for proposal in proposals: #per ogni proposal
            try:
                _, H, W = image_tensor.shape # vedo le dimensioni dell'immagine
                x_min, y_min, x_max, y_max = map(int, proposal)
                x_min, y_min = max(0, x_min), max(0, y_min)
                x_max, y_max = min(W, x_max), min(H, y_max)

                # Controlla se la proposal ha dimensioni valide per l'immagine di partenza -> tecnicamente non si potrebbe eliminare l'if ?
                if x_min < x_max and y_min < y_max:
                    cropped_region = image_tensor[:, y_min:y_max, x_min:x_max]  # Ritaglio

                    # Controlla che il ritaglio non sia vuoto
                    if cropped_region.numel() == 0:
                        print(f"Ritaglio vuoto per proposal: {proposal}. Salto.")
                        continue

                    # Controlla che il tensor sia 3D (C, H, W)
                    if cropped_region.ndim != 3:
                        print(f"Proposal non valida per il ridimensionamento: {proposal}. Salto.")
                        continue

                    resized_region = torch.nn.functional.interpolate(
                        cropped_region.unsqueeze(0), size=output_size, mode='bilinear', align_corners=False
                    ).squeeze(0)  # Ridimensiona

                    processed_proposals.append(resized_region)
            except Exception as e:
                print(f"Errore durante il processamento della proposal: {proposal}. Errore: {e}")

        return processed_proposals  # Lista di tensori delle region proposals


## Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, txt_file, img_dir, coco_json_file, aug=False):
        def generate_id(file_name): #prende il nome.jpg di una immagine e restituisce solo l'identificativo senza prefissi e suffissi
            return file_name.replace('_', '').replace('.jpg', '').replace('img', '')

        with open(txt_file, 'r') as f: #salva le region proposals in un file txt con direttamente le info della cartella
            self.image_paths = [line.strip() for line in f.readlines()] #memorizzo i path delle immagini in una lista

        with open(coco_json_file, 'r') as f: #leggo il file .json - contenente (...) - con coco
            coco_data = json.load(f)

        self.image_annotations = {} #dizionario contenente per ogni immagine una lista di categorie di oggetti presebti

        for annotation in coco_data['annotations']: #uso la sezione annotazioni del file .json per ricavare delle info. sulle immagini del dataset
            image_id = annotation['image_id']
            category_id = annotation['category_id'] # lista di category_id = categorie degli oggetti nell'immagine)

            if image_id not in self.image_annotations: #verifico se l'id dell'immagine è già presente nel dizionario
                self.image_annotations[image_id] = []

            self.image_annotations[image_id].append(category_id)

        self.image_info = {
            int(generate_id(image['file_name'])): image['file_name']
            for image in coco_data['images']
        } #dizionario in cui per ogni nome dell'immagine ottenuta da generate_id(file_name) associa il nome.jpg dell'imagine

        self.base_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ]) # trasformazione di base da applicare a tutte le immagini

        # lasciare momentaneamente in caso di aggiornamenti futuri
        self.aug_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ]) # strasformazione per la data agumentation

        self.aug = aug

    def __len__(self): # ritorna il numero di elementi in self.image_paths -> chi è?
        return len(self.image_paths)

    def __getitem__(self, index):
        img_name = os.path.basename(self.image_paths[index]) #prendo il path dell'immagine da self.image_paths in base all'indice fornito
        img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', '')) #ricavo l'id dell'immagine -> non l'ho già fatto nell'init?

        if img_id not in self.image_info: #vedo dal dizionario self.image_info se l'immagine è contenuta nel file COCO
            raise ValueError(f"Immagine {img_name} non trovata nel file COCO")

        img_path = os.path.join(self.img_dir, img_name) #prendo il path completo dell'immagine unendo il path della cartella con il nome.jpg dell'immagine
        if not os.path.exists(img_path): #se il path non esiste allora lo segnalo
            raise ValueError(f"Immagine non trovata nel percorso: {img_path}")

        image = Image.open(img_path).convert('RGB') #apro l'immagine e ne ricavo le dimensioni
        #original_width, original_height = image.size

        if self.aug: #se la variabile self.aug è alta allora applico la self.aug_transform altrimenti la self.base_transform _> HA SENSO? LE DUE FUNZIONI SONO = !!
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)

        #POTREBBE ESSERCI UN PROBLEMA NELLA CORRISPONDENZA TRA LABLES E REGION PROPOSALS -> perchè in proposals_tensor non ci sono tutte le region proposals perchè alcune vengono scartate
        # -> se la corrispondenza è 1 a 1 allora potrebbe convenire dare a _generate_region_proposals(image) sia le labes che l'immagine? -> non è certo perchè
        # in lables c'è una lista di lable ma non viene specificato dove sono localizzate

        # restituisce un dizionario
        return {
            "regions": image_tensor  # region proposals elaborate, una lista di tensori che rappresentano regioni candidate per il rilevamento
        }

# DataLoader Legacy

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, txt_file, img_dir, coco_json_file, aug=False):
        def generate_id(file_name):
            return file_name.replace('_', '').replace('.jpg', '').replace('img', '')

        with open(txt_file, 'r') as f:
            self.image_paths = [line.strip() for line in f.readlines()]

        self.img_dir = img_dir

        with open(coco_json_file, 'r') as f:
            coco_data = json.load(f)

        self.image_annotations = {}
        self.image_bboxes = {}

        for annotation in coco_data['annotations']:
            image_id = annotation['image_id']
            category_id = annotation['category_id']
            bbox_str = annotation['bbox']
            bbox = list(map(float, bbox_str.strip('[]').split(', ')))

            if image_id not in self.image_annotations:
                self.image_annotations[image_id] = []
                self.image_bboxes[image_id] = []

            self.image_annotations[image_id].append(category_id)
            self.image_bboxes[image_id].append(bbox)

        self.image_info = {
            int(generate_id(image['file_name'])): image['file_name']
            for image in coco_data['images']
        }

        self.base_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ])

        self.aug_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ])

        self.aug = aug    

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_name = os.path.basename(self.image_paths[index])
        img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', ''))
        
        if img_id not in self.image_info:
            raise ValueError(f"Immagine {img_name} non trovata nel file COCO")
    
        img_path = os.path.join(self.img_dir, img_name)
        if not os.path.exists(img_path):
            raise ValueError(f"Immagine non trovata nel percorso: {img_path}")
        
        image = Image.open(img_path).convert('RGB')
        original_width, original_height = image.size
        
        if self.aug:
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)
        
        # Ridimensiona i bounding boxes
        categories = self.image_annotations.get(img_id, [])
        bboxes = self.image_bboxes.get(img_id, [])
        categories = [c for c in categories if isinstance(c, int)]
        if not categories:
            categories = [-1]  # Etichetta speciale per immagini senza annotazioni
        
        scale_x = 320 / original_width
        scale_y = 320 / original_height
        scaled_bboxes = [
            torch.tensor([
                bbox[0] * scale_x,  # x_min
                bbox[1] * scale_y,  # y_min
                bbox[2] * scale_x,  # x_max
                bbox[3] * scale_y   # y_max
            ], dtype=torch.float32)
            for bbox in bboxes
        ] if bboxes else [torch.zeros(4, dtype=torch.float32)]
        
        labels = torch.tensor(categories, dtype=torch.int64)
        proposals_tensor = self._generate_region_proposals(image)
        
        # Chiama match_proposals_with_labels per associare le regioni con le etichette
        matched_proposals, matched_labels = self.match_proposals_with_labels(proposals_tensor, scaled_bboxes, labels)
        
        # Elabora le proposte abbinate
        processed_proposals = self._process_proposals(image_tensor, matched_proposals)
        
        return {
            "image": image_tensor,
            "labels": matched_labels,  # Etichette abbinate alle regioni
            "bboxes": scaled_bboxes,
            "regions": processed_proposals  # Lista di tensori delle region proposals
        }

    def match_proposals_with_labels(self, regions, bboxes, labels, iou_threshold=0.5):
        matched_features = []
        matched_labels = []

        for region in regions:
            # Calcola l'IoU per ciascun bbox
            ious = self.calculate_iou(region, bboxes)
            max_iou = ious.max()
            # MANTENERE PROPORZIONE 1/3 CON 
            if max_iou > iou_threshold:
                # Se IoU > threshold, associa l'etichetta del bbox con massimo IoU
                matched_features.append(region)  # Region proposal
                matched_labels.append(labels[ious.argmax()])  # Etichetta corrispondente
            else:
                # Se nessun bbox corrisponde, classifica come background
                matched_features.append(region)
                matched_labels.append(0)  # 0 indica "background"

        return matched_features, matched_labels

    def calculate_iou(self, region, bboxes):
        # Calcola l'Intersection over Union (IoU) tra la regione proposta e i bounding boxes
        x_min, y_min, x_max, y_max = region
        proposal_area = (x_max - x_min) * (y_max - y_min)

        ious = []
        for bbox in bboxes:
            bx_min, by_min, bx_max, by_max = bbox
            inter_x_min = max(x_min, bx_min)
            inter_y_min = max(y_min, by_min)
            inter_x_max = min(x_max, bx_max)
            inter_y_max = min(y_max, by_max)
            
            # Calcola l'area dell'intersezione
            inter_width = max(0, inter_x_max - inter_x_min)
            inter_height = max(0, inter_y_max - inter_y_min)
            inter_area = inter_width * inter_height
            
            union_area = proposal_area + (bx_max - bx_min) * (by_max - by_min) - inter_area
            ious.append(inter_area / union_area if union_area > 0 else 0)
        
        return torch.tensor(ious)

    def _generate_region_proposals(self, image):
        img_np = np.array(image)
        
        if len(img_np.shape) == 3 and img_np.shape[0] == 3:
            img_np = np.transpose(img_np, (1, 2, 0))  # Da [C, H, W] a [H, W, C]
        elif len(img_np.shape) == 2:
            img_np = np.stack([img_np] * 3, axis=-1)  # Da [H, W] a [H, W, 3]
        elif img_np.shape[2] < 3:
            img_np = np.repeat(img_np, 3, axis=2)  # Da [H, W, 1] a [H, W, 3]
        elif img_np.shape[2] != 3:
            raise ValueError(f"L'immagine ha una forma non valida: {img_np.shape}")
        
        _, regions = selectivesearch.selective_search(img_np, scale=500, sigma=0.9, min_size=10)
        if len(regions) == 0:
            print(f"Warning: Nessuna regione proposta generata per immagine con forma {img_np.shape}.")
        
        proposals = []
        for region in regions:
            x, y, w, h = region['rect']
            if w > 0 and h > 0 and w >= 10 and h >= 10:
                x_max, y_max = min(x + w, img_np.shape[1]), min(y + h, img_np.shape[0])
                proposals.append([x, y, x_max, y_max])
        
        filtered_proposals = self._filter_proposals(proposals, img_np.shape[1], img_np.shape[0])
        
        return filtered_proposals  # Restituisce una lista di coordinate (non ancora tensorizzate)

    def _filter_proposals(self, proposals, img_width, img_height, min_area=100, max_area_ratio=0.8):
        unique_proposals = set(tuple(p) for p in proposals)
        filtered = []
        for x_min, y_min, x_max, y_max in unique_proposals:
            width = x_max - x_min
            height = y_max - y_min
            area = width * height
            if area >= min_area and area <= max_area_ratio * (img_width * img_height):
                filtered.append((x_min, y_min, x_max, y_max))
        return filtered

    def _process_proposals(self, image_tensor, proposals, output_size=(227, 227)):
        processed_proposals = []
        for proposal in proposals:
            try:
                _, H, W = image_tensor.shape
                x_min, y_min, x_max, y_max = map(int, proposal)
                x_min, y_min = max(0, x_min), max(0, y_min)
                x_max, y_max = min(W, x_max), min(H, y_max)
    
                # Controlla se la proposal è valida
                if x_min < x_max and y_min < y_max:
                    cropped_region = image_tensor[:, y_min:y_max, x_min:x_max]  # Ritaglio
                    
                    # Controlla che il ritaglio non sia vuoto
                    if cropped_region.numel() == 0:
                        print(f"Ritaglio vuoto per proposal: {proposal}. Salto.")
                        continue
                    
                    # Controlla che il tensor sia 3D (C, H, W)
                    if cropped_region.ndim != 3:
                        print(f"Proposal non valida per il ridimensionamento: {proposal}. Salto.")
                        continue
                    
                    resized_region = torch.nn.functional.interpolate(
                        cropped_region.unsqueeze(0), size=output_size, mode='bilinear', align_corners=False
                    ).squeeze(0)  # Ridimensiona
                    
                    processed_proposals.append(resized_region)
            except Exception as e:
                print(f"Errore durante il processamento della proposal: {proposal}. Errore: {e}")
    
        return processed_proposals  # Lista di tensori delle region proposals

In [ ]:
def collate_fn(batch):
    images = []
    labels = []
    bboxes = []
    regions = []

    for sample in batch:
        images.append(sample['image'])  # Le immagini sono già tensori
        labels.append(torch.tensor(sample['labels'], dtype=torch.int64))  # Convertiamo in tensore
        bboxes.append(sample['bboxes'])  # Bboxes potrebbero essere già tensori
        regions.append(sample['regions'])  # Le region proposals potrebbero essere già tensori

    # Stacking delle immagini e concatenamento delle etichette
    images = torch.stack(images, dim=0)
    labels = torch.cat(labels, dim=0)  # Ora labels sono tensori, possiamo concatenarle
    bboxes = [torch.stack(b, dim=0) if len(b) > 0 else torch.zeros(1, 4) for b in bboxes]  # Gestione delle bounding boxes
    regions = [torch.stack(r, dim=0) if len(r) > 0 else torch.zeros(1, 4) for r in regions]  # Gestione delle region proposals

    return {
        'image': images,
        'labels': labels,
        'bboxes': bboxes,
        'regions': regions
    }

In [ ]:
# Creazione dei dataset
train_dataset = CustomDataset(train_txt_pth, save_images_fldr_pth, new_coco_json_pth, aug=True)
valid_dataset = CustomDataset(val_txt_pth, save_images_fldr_pth, new_coco_json_pth, aug=False)  
test_dataset = CustomDataset(test_txt_pth, save_images_fldr_pth, new_coco_json_pth, aug=False)  

# Creazione dei DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

## Check DataLoader

In [ ]:
# Numero totale di campioni per ogni DataLoader
train_size = len(train_loader.dataset)
val_size = len(val_loader.dataset)
test_size = len(test_loader.dataset)

# Numero di batch per ogni DataLoader
train_batches = len(train_loader)
val_batches = len(val_loader)
test_batches = len(test_loader)

# Visualizza i risultati
print(f"Numero totale di elementi nel train_loader: {train_size}")
print(f"Numero totale di batch nel train_loader: {train_batches}")
print(f"Numero totale di elementi nel val_loader: {val_size}")
print(f"Numero totale di batch nel val_loader: {val_batches}")
print(f"Numero totale di elementi nel test_loader: {test_size}")
print(f"Numero totale di batch nel test_loader: {test_batches}")

# Somma totale degli elementi nei DataLoader
total_elements = train_size + val_size + test_size
print(f"Numero totale di elementi in tutti i DataLoader: {total_elements}")

In [ ]:
def check_txt_vs_json(txt_paths, coco_data):
    """
    Controlla se le immagini del JSON sono presenti in almeno uno dei file TXT.
    
    Args:
        txt_paths (list): Lista di percorsi ai file TXT.
        coco_data (dict): Dati in formato COCO.
    """
    # Estrai i nomi delle immagini dal JSON
    image_names = [image['file_name'] for image in coco_data['images']]
    
    # Inizializza un set per contenere tutte le immagini presenti nei TXT
    txt_image_names = set()
    
    # Leggi i nomi delle immagini da ciascun file TXT
    for txt_path in txt_paths:
        with open(txt_path, 'r') as f:
            txt_image_names.update(os.path.basename(line.strip()) for line in f.readlines())
    
    # Trova le immagini presenti nel JSON ma non in nessuno dei TXT
    missing_in_txts = [name for name in image_names if name not in txt_image_names]
    
    # Verifica e stampa i risultati
    print("\nControllo completato:")
    if missing_in_txts:
        print(f"Errore: le seguenti immagini non sono presenti in nessuno dei file TXT forniti:\n{missing_in_txts}")
    else:
        print("Tutte le immagini del JSON sono presenti in almeno uno dei file TXT.")

In [ ]:
# Conta il numero di immagini nel JSON
num_images = len(coco_data['images'])
print(f"Numero di immagini nel JSON: {num_images}")

In [ ]:
# Estrai i nomi delle immagini dal JSON
image_names = [image['file_name'] for image in coco_data['images']]

# Stampa i primi 5 nomi delle immagini nel JSON
print(f"Primi 5 nomi delle immagini nel JSON: {image_names[:5]}")   

In [ ]:
def count_lines_in_txt(txt_paths):
    """
    Conta il numero di righe in ciascun file TXT dato il percorso e calcola la somma totale delle righe.
    
    Args:
        txt_paths (list): Lista di percorsi ai file TXT.
    """
    total_lines = 0  # Variabile per accumulare il numero totale di righe
    
    for txt_path in txt_paths:
        try:
            # Apri il file e conta le righe
            with open(txt_path, 'r') as f:
                num_lines = sum(1 for line in f)
            total_lines += num_lines  # Aggiungi il numero di righe del file al totale
            print(f"Numero di righe nel file {txt_path}: {num_lines}")
        except FileNotFoundError:
            print(f"Errore: il file {txt_path} non è stato trovato.")
        except Exception as e:
            print(f"Errore nel leggere il file {txt_path}: {e}")
    
    # Stampa la somma totale delle righe
    print(f"\nSomma totale delle righe in tutti i file: {total_lines}")

In [ ]:
txt_paths = [train_txt_pth, val_txt_pth, test_txt_pth]
count_lines_in_txt(txt_paths)

In [ ]:
check_txt_vs_json(txt_paths, coco_data)

In [ ]:
def check_dataloader(loader):
    """
    Funzione per controllare il comportamento di un DataLoader.
    Visualizza alcune immagini insieme alle loro annotazioni per verificare il corretto funzionamento.

    Args:
        loader (DataLoader): Il DataLoader da verificare.
    """
    for batch_idx, batch in enumerate(loader):
        print(f"Batch {batch_idx + 1}:")
        
        # Estrai i dati dal dizionario
        images = batch["image"]
        labels = batch["labels"]
        bboxes = batch["bboxes"]
        regions = batch["regions"]

        # Stampa le shape e le dimensioni dei dati
        print(f"  Shape delle immagini (batch): {images.shape}")
        print(f"  Numero di etichette nel batch: {len(labels)}")

        # Controlla se il batch è vuoto
        if images.size(0) == 0:
            print("Batch vuoto. Procedo con il batch successivo.")
            continue

        # Scegli un'immagine casuale dal batch
        random_index = random.randint(0, images.size(0) - 1)
        image = images[random_index].permute(1, 2, 0).numpy()  # Da [C, H, W] a [H, W, C]

        # Visualizza l'immagine
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
        ax.imshow(image)
        ax.axis("off")
        ax.set_title(f"Immagine nel batch {batch_idx + 1}, indice {random_index}")

        # Etichette
        label = labels[random_index]
        label_info = label.tolist() if isinstance(label, torch.Tensor) else label
        print(f"  Etichette: {label_info}")

        # Bounding boxes
        bbox = bboxes[random_index]
        for box in bbox:
            if isinstance(box, torch.Tensor):
                box = box.tolist()  # Converte il bounding box in lista
    
            # Converti da [x_min, y_min, width, height] a [x_min, y_min, x_max, y_max]
            x_min, y_min, width, height = box
            x_max = x_min + width
            y_max = y_min + height
    
            # Opzionale: Verifica dei limiti dell'immagine
            H, W, _ = image.shape
            x_min, y_min = max(0, x_min), max(0, y_min)
            x_max, y_max = min(W, x_max), min(H, y_max)
    
            # Disegna il bounding box
            rect = plt.Rectangle((x_min, y_min), width, height,
                                  edgecolor='green', facecolor='none', linewidth=1.5)
            ax.add_patch(rect)

        # Visualizza alcune region proposals come immagini separate
        region_proposals = regions[random_index]
        print(f"  Numero di region proposals: {len(region_proposals)}")

        fig, axs = plt.subplots(1, min(5, len(region_proposals)), figsize=(15, 5))
        for i, proposal in enumerate(region_proposals[:5]):
            proposal_image = proposal.permute(1, 2, 0).numpy()  # Da [C, H, W] a [H, W, C]
            axs[i].imshow(proposal_image)
            axs[i].axis("off")
            axs[i].set_title(f"Region Proposal {i + 1}")
        plt.show()

        break  # Mostra solo il primo batch

In [ ]:
# Esegui il controllo per i dataloader
check_dataloader(train_loader)
check_dataloader(val_loader)
check_dataloader(test_loader)